# Proyecto de programación Deep Vision

## 0. Carga de librerías y ajustes iniciales

In [ ]:
# Google
from google.colab import drive, files

# Numpy
import numpy as np
from numpy.random import seed

# OpenCV to process images
import cv2

# Directory indexes
from os import listdir

# Random
import random

# Matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, BatchNormaliz
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Sklearn
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Ploteado del esquema gráfico del modelo
from keras.utils.vis_utils import plot_model

### Configuración y carga de seeds

In [ ]:
# Semilla para numpy
seed(7)

# Semilla para tensorflow
tf.random.set_seed(8)

### Conexión y directorio de trabajo en google drive

In [ ]:
drive.mount('/content/drive')

BASE_FOLDER = '/content/drive/MyDrive/07_MIAR_RNDL_CONV2/ProyectoProgramacion/'

### 0.1. Definición de funciones auxiliares y utilidades

#### Función para el ploteado de curva de aprendizaje

In [ ]:
def plot_learning_curves(H,epochs):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc")
    plt.title("Entrenamiento: Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

#### Función diseñada para extraer imágenes y sus correspondientes etiquetas

In [ ]:
# Función de conversión a array NumPy
def toNumPyArray_x_y(image_ds):
    x = []
    y = []
    #for image, label in tfds.as_numpy(image_ds):
    #for image, label in image_ds.as_numpy_iterator():
    for image, label in image_ds:
        x.append(image.numpy())
        y.append(label.numpy())
    x = np.array(x)
    y = np.array(y)
    return x, y

#### Función para ploteado de diagrama de arquitectura

In [ ]:
# Función para mostrar gráfico de arquitectura de red
def plot_schema(model_name, file_name='model_plot.png', shapes=True, layer_names=True):
    return plot_model(model=model_name,
        to_file=file_name,
        show_shapes=shapes,
        show_layer_names=layer_names)

## 1. Carga del conjunto de datos (desde Kaggle)

In [ ]:
# Verificación de la versión de la API de Kaggle en Colab
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
# Seleccionamos y cargamos el API de Kaggle descargado (kaggle.json)
files.upload()

# Creamos directorio donde guardar el fichero kaggle.json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargamos dataset de Kaggle
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

# Creamos directorio para descomprimir los datos
!mkdir my_dataset

# Descomprimimos datos
!unzip the-nature-conservancy-fisheries-monitoring.zip -d my_dataset

# descomprimo en mi directorio google drive
%%capture
!unzip my_dataset/train.zip -d my_dataset

#!unzip my_dataset/test_stg1.zip